### bq 데이터를 kafka에 저장

In [1]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
LIMIT = 10

In [2]:
BOOTSTRAP_SERVERS = "datafabric-kafka-kafka-bootstrap.kafka-farm.svc.cluster.local:9092"
TOPICS = 'test'
CONSUMER_GROUP = 'test-datafabric'

In [9]:
import traceback
from pydatafabric.gcp import bq_to_pandas

df = bq_to_pandas(f"""
    select review_id as key, comments as value
    from  `{PROJECT}.{DATASET}.{TABLE}`
    where comments != ''
    limit {LIMIT}
""")

unsupported operand type(s) for /: 'NoneType' and 'int'


Downloading: 100%|██████████| 10/10 [00:00<00:00, 12.50rows/s]


In [4]:
df.head(10)

,key,value
0,2208281244313807,샤인머스캣 너무 비싸요
1,2208280027362716,요즘 계속 사게 되네요
2,2208281618561112,주말 특가로 저령하게 구입했는데 당도도 아주 좋아요
3,2208280253124586,당도도 높고 너무 맛있게먹었어요
4,2208281913283031,.껍질이 두꺼워 실망했어요 먹을부분은 작아지니까~ 맛은 달콤하니 좋아여. 여름 끝...
5,2208280102419526,가격도싸고달고맛있어요.
6,2208281643108429,포도하면 송산포도가 젤이예여
7,2208280202528223,좋아해서 구매했는데.맛있습니다.
8,2208280032177490,올해 먹은 포도 중에 최고
9,2208280956359149,조아요 조아요 조아요 조아요 조아요 조아요


In [5]:
print(df.dtypes)

key      object
value    object
dtype: object


### Send Kafka

In [6]:
!python -m pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from kafka import KafkaProducer
from json import dumps

producer = KafkaProducer(acks="all",
                         compression_type='gzip',
                         bootstrap_servers=BOOTSTRAP_SERVERS,
                         value_serializer=lambda x: dumps(x).encode('utf-8')
                         )

try:
    for n in range(100):
        producer.send(TOPICS, value=df.to_json(force_ascii=False, orient = 'records', indent=4))
        producer.flush()
except:
    traceback.print_exc()

### Reading Data from Kafka

In [10]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer(
    TOPICS,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    group_id=CONSUMER_GROUP,        # 그룹핑하여 토픽 지정할 수 있다 > 같은 컨슈머로 작업
    auto_offset_reset="latest",   # 어디서부터 값을 읽어올지 (earlest 가장 처음 latest는 가장 최근)
    enable_auto_commit=False,
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    max_poll_records = 10,
    consumer_timeout_ms = 1000,
)

In [33]:
for message in consumer:
    print("Topic: %s, Partition: %d, Offset: %d, Key: %s, Value: %s" % (
        message.topic, message.partition, message.offset, message.key, message.value
    ))

In [67]:
# # Create Topic Partitions
# from kafka import KafkaAdminClient
# from kafka.admin.new_partitions import NewPartitions

# client = KafkaAdminClient(bootstrap_servers=BOOTSTRAP_SERVERS)

# rsp = client.create_partitions({
#     TOPICS: NewPartitions(4)
# })
# print(rsp)

CreatePartitionsResponse_v1(throttle_time_ms=0, topic_errors=[(topic='test', error_code=0, error_message=None)])
